## Tensorflow Tutorial
Tensorflow is an open source deep learning library by Google.It is one of the most popular library out there.
This tutorial shows a convolution neural network applied to one of the most famous dataset of deep learning.MNIST data set will be used.It contains handwritten digits from 0-9.Tensorflow tutorials have mnist dataset preloaded.Recurrent neural network are widely used in natural language processing,chatbots,language translatiom,etc.
Tensorflow has an amazing visualization tool known as tensorboard.

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("MNIST_data/", one_hot=True) #one hot true does one hot encoding
tf.reset_default_graph()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


Epoch is one round of forward propogation and one round of backpropogation.There are total 10 labels(0-9) and there are a total of 10,000 images.

In [2]:
epoch=10
display_epoch = 1
logs_path = 'mnist'
n_classes = 10  #Total 10 digits
hm_epochs = 3
batch_size = 100
n_input = 28
timesteps = 28
rnn_size = 500

Input data are taken as placeholders.These palceholders are feeded with dictionary.For x placeholder it expects input of [somebatchsize,28,28] and y expects as [somebatchsize,10].The reason of giving none is that we can change our batch size.

In [3]:
with tf.name_scope('inputs') as inputs:
        x = tf.placeholder(tf.float32, shape=(None,timesteps,n_input), name='x')
        y = tf.placeholder(tf.float32, shape=(None,10), name='y')

To initialize weights we use xavier initializer.Xavier initializer chooses values in random from a gaussian distribution based on the input and output layer.

In [4]:
def weights_xavi(shape, xavier_params = (None, None)):
    with tf.variable_scope('Weights'):
        (fan_in, fan_out) = xavier_params
        low = -4*np.sqrt(6.0/(fan_in + fan_out)) # {sigmoid:4, tanh:1} 
        high = 4*np.sqrt(6.0/(fan_in + fan_out))
        return tf.Variable(tf.random_uniform(shape, minval=low, maxval=high, dtype=tf.float32))

Neural net is a function that takes two inputs as parameters.The first parameter la expects the input layer/previous layer.The second parameter takes dimension of the previous layer and the dimension of the current layer.The function does a matrix multiplication on previous layer with the current layer weights w and then sums with bias b.

In [5]:
def neuralnet(la,shape=[None,None]):
    with tf.name_scope('neuralnet'):
        w=weights_xavi(shape,xavier_params=(shape[0],shape[1]))
        b=weights_xavi([shape[1],],xavier_params=(0,shape[1]))
        act=tf.add(tf.matmul(la,w),b)
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return act

Neural net is a function that takes two inputs as parameters.The first parameter la expects the input image/previous layer.The second parameter takes dimension as [kernel size,kernel size,previous no of channels,current no of channels].The function does convolution operation on previous layer with the current layer weights w and then sums with bias b. 

RNN cell takes in input x and gives two outputs. The state outputs become inputs along with x in the next time step. This state outputs help in preserving the memory of the cell. However during backpropogation RNN face a problem of vanishing gradient. To solve this problem LSTM cell are used. To know more about LSTM visit the blog :- http://colah.github.io/posts/2015-08-Understanding-LSTMs/ 

In [6]:
def rnn_cell():
    return tf.contrib.rnn.BasicRNNCell(rnn_size)

In [7]:
def lstm_cell():
    return tf.contrib.rnn.BasicLSTMCell(rnn_size,forget_bias=1.0)


In our model the inputs are unstacked according to timesteps and are fed to the LSTM cell. The output from LSTM is then multiplied with weights and biases to give the final output.

In [8]:
def RNN(la,shape=[None,None]):
    with tf.variable_scope('RNN'):
        la =tf.unstack(la, timesteps, 1)
        w=weights_xavi(shape,xavier_params=(shape[-2],shape[-1]))
        b=weights_xavi([shape[-1],],xavier_params=(0,shape[-1]))
        lscell=lstm_cell()
        output,states=tf.contrib.rnn.static_rnn(lscell,la,dtype=tf.float32)
        act=tf.add(tf.matmul(output[-1],w),b)
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return act

The loss function used is softmax_cross_entropy and the optimization function used is Adam.Adam optimizer works better than Gradient Descent algorithm.In tensorflow a graph is built and executed using tf.Session(). To execute each function and variable sess.run() is used.

In [9]:
with tf.name_scope('Model'): 
    pred = RNN(x,shape=[500,10])                      #we get a output of our network
with tf.name_scope('Loss'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
    tf.summary.scalar("Loss",cost)         #we find the cost using soft cross entopy.
with tf.name_scope('ADAM'):
    optimizer = tf.train.AdamOptimizer().minimize(cost)  #We use Adam Optimizer
with tf.name_scope('Accuracy'):
    correct = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))   #Calculate Accuracy
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
    tf.summary.scalar('accuracy', accuracy)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()        
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())       #Initializes All the variables
    summary_writer = tf.summary.FileWriter(logs_path)  #WritesTheGraphForTensorboard
    summary_writer.add_graph(graph=sess.graph)        
    gstep=1
    for i in range(epoch):
            epoch_loss = 0             #calculate loss
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                epoch_x=epoch_x.reshape((batch_size,timesteps,n_input))#Selects input and label
                #The optimizer,Cost,Tensorboard Function are Run.
                _, c,summary = sess.run([optimizer, cost,merged_summary_op], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c
                summary_writer.add_summary(summary,gstep) #Writes Summary
                gstep+=1

            print('Epoch', i, 'completed out of',epoch,'loss:',epoch_loss)
    print('Accuracy:',accuracy.eval({x:mnist.test.images.reshape((-1,timesteps,n_input)), y:mnist.test.labels}))

Epoch 0 completed out of 10 loss: 207.914453328
Epoch 1 completed out of 10 loss: 56.6717027565
Epoch 2 completed out of 10 loss: 38.6725350488
Epoch 3 completed out of 10 loss: 29.4090174411
Epoch 4 completed out of 10 loss: 24.812942869
Epoch 5 completed out of 10 loss: 18.9755650715
Epoch 6 completed out of 10 loss: 17.1648511103
Epoch 7 completed out of 10 loss: 14.9044732371
Epoch 8 completed out of 10 loss: 13.2384526202
Epoch 9 completed out of 10 loss: 11.3864813036
Accuracy: 0.9875


In [10]:
print("Run the command line:\n" \
          "--> tensorboard --logdir='mnist' " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")

Run the command line:
--> tensorboard --logdir='mnist' 
Then open http://0.0.0.0:6006/ into your web browser


# Tensorboard Graph Visualization
<img src ='images/rnn.png'>

## Accuracy And Losses
<img src='images/arn.png'>
We can see that accuracy increases and the loss decreases.Thus our network is working properly.